In [13]:
import os
import pandas as pd
import requests
import geopy.distance
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
df = pd.read_csv('../data/geocoded_addresses.csv')
df

,Unnamed: 0,OBJECTID,CaseNum,DecedentAge,Mode,SubManner,Gender,Race,DeathDate,DeathYear,...,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,partial_match,plus_code.compound_code,plus_code.global_code,postcode_localities,closest_node,node_distance
0,0,1,19-00010,29.0,Driver,Accident (MVA),Male,African American,1/2/2019 12:00:00 AM,2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,19-00054,57.0,Driver,Accident (MVA),Male,"White, Italian",1/4/2019 12:00:00 AM,2019,...,38.514745,-121.079403,38.512047,-121.082101,NaN,NaN,NaN,NaN,9.045171e+07,18.116500
2,2,3,19-00066,53.0,Driver,Accident (MVA),Female,"White, Spanish",1/5/2019 12:00:00 AM,2019,...,38.511903,-121.412424,38.509206,-121.415122,NaN,NaN,NaN,NaN,9.069184e+07,1.250697
3,3,4,19-00084,58.0,Driver,Accident (MVA),Male,White,1/6/2019 12:00:00 AM,2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,6,19-00197,57.0,Pedestrian,Accident (MVA),Male,Mexican,1/10/2019 12:00:00 AM,2019,...,38.662256,-121.353828,38.659558,-121.356526,NaN,NaN,NaN,NaN,7.779937e+09,2.734753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1137,1137,1261,24-04009,32.0,Other,Accident (MVA-Criminal),Female,African American,8/30/2024 12:00:00 AM,2024,...,38.571109,-121.464508,38.568411,-121.467206,NaN,NaN,NaN,NaN,9.045134e+07,76.610684
1138,1138,1262,24-04019,86.0,Other,Accident (MVA-Criminal),Female,White,8/30/2024 12:00:00 AM,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1139,1139,1264,24-04082,23.0,Other,Accident (MVA),Male,White,9/3/2024 12:00:00 AM,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1140,1140,1266,24-04117,32.0,Driver,Accident (MVA),Male,Hispanic,9/5/2024 12:00:00 AM,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
nodes = pd.read_csv('../data/nodes.csv')
nodes

,osmid,y,x,street_count,ref,highway,geometry
0,90242839,38.306560,-121.572749,3,NaN,NaN,POINT (-121.572749 38.30656)
1,90242879,38.313134,-121.572265,1,NaN,NaN,POINT (-121.572265 38.313134)
2,90242884,38.305919,-121.572372,3,NaN,NaN,POINT (-121.572372 38.305919)
3,90242894,38.306250,-121.570508,1,NaN,NaN,POINT (-121.5705075 38.3062504)
4,90243026,38.105461,-121.593364,3,NaN,NaN,POINT (-121.593364 38.105461)
...,...,...,...,...,...,...,...
49112,11069939783,38.253089,-121.283397,3,NaN,NaN,POINT (-121.2833972 38.2530893)
49113,11069939785,38.253734,-121.286258,3,NaN,NaN,POINT (-121.2862577 38.2537344)
49114,11069939786,38.253768,-121.283405,3,NaN,NaN,POINT (-121.2834046 38.2537675)
49115,11990500052,38.262552,-121.264004,3,NaN,NaN,POINT (-121.2640039 38.2625518)


In [5]:
edges = pd.read_csv('../data/edges.csv')
edges

/var/folders/s_/ms4snwvs2b59mw5fvp44bvww0000gq/T/ipykernel_34917/1363276170.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  edges = pd.read_csv('../data/edges.csv')


,u,v,key,osmid,name,highway,oneway,reversed,length,geometry,lanes,ref,bridge,maxspeed,access,junction,tunnel,width
0,90242839,90242884,0,10527526,River Road,secondary,False,False,78.501,"LINESTRING (-121.572749 38.30656, -121.572372 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90242839,90242879,0,10496968,NaN,residential,False,False,921.563,"LINESTRING (-121.572749 38.30656, -121.5719715...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,90242839,90600174,0,10527526,River Road,secondary,False,True,1414.237,"LINESTRING (-121.572749 38.30656, -121.57285 3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90242879,90242839,0,10496968,NaN,residential,False,True,921.563,"LINESTRING (-121.572265 38.313134, -121.571880...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,90242884,90242839,0,10527526,River Road,secondary,False,True,78.501,"LINESTRING (-121.572372 38.305919, -121.572749...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122089,11069939786,11069939785,0,1192494762,NaN,residential,False,True,249.157,"LINESTRING (-121.2834046 38.2537675, -121.2862...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122090,11990500052,90434645,0,1293472696,Simmerhorn Road,tertiary,False,False,2.715,"LINESTRING (-121.2640039 38.2625518, -121.2639...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122091,11990500052,90434643,0,10508836,Cherokee Lane,residential,False,True,102.877,"LINESTRING (-121.2640039 38.2625518, -121.2640...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122092,11990500052,90473005,0,1293472696,Simmerhorn Road,tertiary,False,True,1623.086,"LINESTRING (-121.2640039 38.2625518, -121.2649...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
nodes = nodes.loc[nodes['street_count'] > 1]
nodes

,osmid,y,x,street_count,ref,highway,geometry
0,90242839,38.306560,-121.572749,3,NaN,NaN,POINT (-121.572749 38.30656)
2,90242884,38.305919,-121.572372,3,NaN,NaN,POINT (-121.572372 38.305919)
4,90243026,38.105461,-121.593364,3,NaN,NaN,POINT (-121.593364 38.105461)
6,90243790,38.160321,-121.674078,3,NaN,NaN,POINT (-121.6740776 38.1603212)
7,90245899,38.280267,-121.457328,3,NaN,NaN,POINT (-121.4573283 38.2802666)
...,...,...,...,...,...,...,...
49111,11069939782,38.253067,-121.286246,3,NaN,NaN,POINT (-121.2862455 38.2530672)
49112,11069939783,38.253089,-121.283397,3,NaN,NaN,POINT (-121.2833972 38.2530893)
49113,11069939785,38.253734,-121.286258,3,NaN,NaN,POINT (-121.2862577 38.2537344)
49114,11069939786,38.253768,-121.283405,3,NaN,NaN,POINT (-121.2834046 38.2537675)


In [19]:
def get_bounding_box(lat, lon, box_size_meters=200):

    d = geopy.distance.geodesic(meters=box_size_meters / 2)
    north, south = d.destination((lat, lon), 0).latitude, d.destination((lat, lon), 180).latitude
    east, west = d.destination((lat, lon), 90).longitude, d.destination((lat, lon), 270).longitude
    
    return north, south, east, west

def fetch_satellite_image(node):
    mapbox_api_key = os.environ['MAPBOX_API_KEY']
    node_id = node['osmid']
    lat = node['y']
    lon = node['x']
    north, south, east, west = get_bounding_box(lat, lon, box_size_meters=92)
    width = 640
    height = 640
    url = f"https://api.mapbox.com/styles/v1/mapbox/satellite-v9/static/[{west},{south},{east},{north}]/{width}x{height}?access_token={mapbox_api_key}"
    response = requests.get(url)

    if response.status_code == 200:
        with open(f'../data/images/satellite/{node_id}.jpg', 'wb') as f:
            f.write(response.content)

for _, node in nodes.sample(10).iterrows():
    fetch_satellite_image(node)


In [29]:
for _, node in nodes.loc[nodes['osmid'].apply(lambda x: x in df.loc[df['Mode']=='Pedestrian','closest_node'].values)].sample(10).iterrows():
    fetch_satellite_image(node)